# Document Service

This notebook forms the core of our document service. It showcases how we are going to simplify our document intelligence application using Lakebase and Serverless jobs. This is tested on Serverless Version 3 - it takes a single file or a directory and parses all the files directly into an append operation on a postgres table. We can then get embeddings and use pgvector as the backend with a langgraph Agent.

We use our Databricks user IDs as the main entry point into the workflow and authentication

In [0]:
%pip install databricks-langchain
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
me = w.current_user.me()
print(me.id)  # This is your Databricks user ID
print(me.user_name) 
USER_ID = me.id

7873535765378608
scott.mckean@databricks.com


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# to be parameterized
volume_path = '/Volumes/shm/default/raw_pdfs/73eb2c4f_3c424fe06ddfe8ce_1755574069.pdf'
embedding_endpoint = 'databricks-gte-large-en'

We use ai_parse_document in a serverless job as our document processing service. This could be any isolated microservice and has lots of room for optimization, but ai_parse_document does a pretty good job and can handle lots of file types

In [0]:
parsed_df = (
    spark.read.format("binaryFile")
    .load(volume_path)
    .withColumn("user_id", lit(USER_ID))
    .select(
        col("path"),
        col("user_id"),
        expr("ai_parse_document(content)").alias("parsed")
    )
    .withColumn(
        "parsed_json",
        parse_json(col("parsed").cast("string"))
    )
    .select(
        col("path"),
        col("user_id"),
        expr("parsed_json:document:pages").alias("pages"),
        expr("parsed_json:document:elements").alias("elements"),
        expr("parsed_json:document:_corrupted_data").alias("_corrupted_data")
    )
)

To get something simple and working, I propose that we simply chunk each page for now. We can work on refining the chunking strategy in this job, but this gives a good starting point. We even wrap the embedding call here for better horizontal scalability.

In [0]:
from pyspark.sql.functions import from_json, explode, col, concat_ws, lit
from pyspark.sql.types import ArrayType, StructType, StructField, IntegerType, StringType

# Define schema for pages based on provided example
page_schema = StructType([
    StructField("content", StringType()),
    StructField("footer", StringType()),
    StructField("header", StringType()),
    StructField("id", IntegerType()),
    StructField("page_number", IntegerType())
])

chunked_pages = (
    parsed_df
    .withColumn(
        "pages_array",
        from_json(
            col("pages").cast("string"),
            ArrayType(page_schema)
        )
    )
    .withColumn(
        "page_chunk",
        explode(col("pages_array"))
    )
    .select(
        col("path"),
        col("user_id"),
        col("page_chunk.id").cast("string").alias("page_id"),
        concat_ws(
            "\n",
            concat_ws("", lit("Content: ["), col("page_chunk.content"), lit("]")),
            concat_ws("", lit("Footer: ["), col("page_chunk.footer"), lit("]")),
            concat_ws("", lit("Header: ["), col("page_chunk.header"), lit("]")),
            concat_ws("", lit("ID: ["), col("page_chunk.id").cast("string"), lit("]")),
            concat_ws("", lit("Page Number: ["), col("page_chunk.page_number").cast("string"), lit("]"))
        ).alias("text")
    )
    .withColumn("embedding", expr(f"ai_query('{embedding_endpoint}', 'text')"))
)

display(chunked_pages)

path user_id page_id text embedding dbfs:/Volumes/shm/default/raw_pdfs/73eb2c4f_3c424fe06ddfe8ce_1755574069.pdf 7873535765378608 0 Content: [# Sample Company
## Income Statement (Service)
For the Year Ended September 30, 2021

| Category | Details | Amount | Total |
|-------------------------|------------------|--------|-------|
| Service revenue | | | $2,750|
| Operating Expenses: | Depreciation expense| 100 | |
| | Wages expenses | 1,200 | |
| | Supplies expenses| 60 | |
| | Total operating expenses | | 1,360 |
| Operating Income | | | 1,390 |
| Other Item: | Interest expense | 40 | |
| | Pretax income | | 1,350 |
| | Income tax expense | 405 | |
| Net income | | | $945 |

# Sample Company
## Statement of Retained Earnings
For the Year Ended September 30, 2021

| Description | Retained Earnings |
|-------------------------|--------------------|
| Balance, October 1, 2020| $820 |
| Net income | 945 |
| Dividends declared | (500) |
| Balance, September 30, 2021 | $1,265 |]
Footer: []
Header: []
ID: [0]
Page Number: [] List(-0.497802734375, -0.27978515625, -0.5244140625, -0.384033203125, 0.287353515625, 0.314697265625, 0.3779296875, -0.4404296875, 0.38330078125, 0.0308685302734375, 0.61279296875, -0.495849609375, -0.43017578125, -0.23291015625, -0.1878662109375, -0.30419921875, -0.26611328125, -0.6103515625, 0.393310546875, -0.51953125, -0.271240234375, 0.2457275390625, 0.339599609375, -0.80419921875, 1.1513671875, 0.343017578125, -1.1865234375, -0.448974609375, 0.373291015625, -0.0706787109375, -0.343994140625, 0.209716796875, -0.56689453125, -0.254638671875, -0.0633544921875, 0.76513671875, -0.09527587890625, -0.367431640625, -0.1612548828125, -0.04058837890625, -0.859375, -0.63671875, -0.270263671875, -0.4521484375, -0.7021484375, -0.12103271484375, 0.11053466796875, 0.018829345703125, -0.152099609375, 0.98828125, -0.9365234375, 0.12225341796875, -0.4453125, -0.1878662109375, 0.404052734375, 0.431884765625, 0.408203125, -0.8359375, 0.5751953125, 0.2254638671875, -0.669921875, 0.1590576171875, -0.272216796875, -0.32666015625, 0.2347412109375, 0.74609375, -0.28125, -0.45263671875, -1.287109375, 0.476318359375, -0.2086181640625, 0.29150390625, 0.1982421875, -1.0224609375, -0.193603515625, -0.127685546875, -0.9677734375, 0.2734375, -0.6533203125, 1.33984375, -0.3984375, -0.1903076171875, -0.28759765625, -1.1533203125, -0.7861328125, 1.205078125, 0.01201629638671875, 0.0931396484375, -0.1705322265625, 0.5478515625, 0.1138916015625, -0.39013671875, -0.03131103515625, 0.15234375, -0.0243072509765625, -0.0467529296875, -0.2076416015625, -0.61767578125, -0.499267578125, -0.0048065185546875, -0.024810791015625, -0.5087890625, -0.76416015625, 0.1258544921875, 0.466552734375, 0.287841796875, -0.5615234375, 0.423095703125, 0.1793212890625, 0.055511474609375, 0.52490234375, 0.35009765625, -0.0191802978515625, 0.052032470703125, 0.163818359375, 0.79052734375, 0.351318359375, -0.4140625, -0.69287109375, 0.12109375, -0.642578125, 0.947265625, 0.67138671875, -0.385986328125, 0.66064453125, 0.26171875, 0.0660400390625, -0.2568359375, -0.36181640625, 0.73779296875, 0.1829833984375, -0.353271484375, -0.0362548828125, -0.470703125, -0.040985107421875, 0.022247314453125, -0.826171875, 0.33984375, 0.86181640625, 0.85498046875, 0.297119140625, -0.6904296875, 0.371826171875, -0.609375, 0.779296875, 0.1756591796875, -0.370361328125, 0.580078125, -0.11126708984375, 0.2412109375, -0.76171875, 0.22314453125, -0.256103515625, 0.1400146484375, 0.00848388671875, 0.04290771484375, -0.10357666015625, -0.437744140625, -0.671875, -1.326171875, -0.85791015625, -0.7685546875, -0.2568359375, 0.37841796875, -0.26904296875, 1.09375, -0.9912109375, -0.5361328125, -0.02490234375, 0.56689453125, -0.6845703125, 0.36376953125, -0.057952880859375, -0.2578125, -0.0341796875, -0.32421875, -0.054290771484375, 0.1787109375, 0.393798828125, -0.66455078125, -0.10198974609375, -1.3447265625, -0.44482421875, -0.5146484375, 0.58935546875, 0.379150390625, 0.333984375, 0.02647399902

In [0]:
chunked_pages_pd = chunked_pages.toPandas()
chunked_pages_pd['embedding'] = chunked_pages_pd['embedding'].apply(lambda x: list(x))

## Postgres Connection
Now we are going to move our chunks into postgres. First we create our chunks table, next we read into Pandas and write chunks into Postgres. This gets around shuttling larger tables quite nicely, and we can bolster the connection and database for horizontal scalability (e.g. https://learn.microsoft.com/en-us/azure/databricks/oltp/query/notebook)

In [0]:
import psycopg2

from databricks.sdk import WorkspaceClient
import uuid

w = WorkspaceClient()

instance_name = "shm"

instance = w.database.get_database_instance(name=instance_name)
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[instance_name])

def connect_to_pg():
  conn = psycopg2.connect(
      host=instance.read_write_dns,
      dbname="databricks_postgres",
      user=me.user_name,
      password=cred.token,
      sslmode="require"
  )
  return conn

In [0]:
table_ddl = """
CREATE TABLE IF NOT EXISTS parsed_pages (
    path TEXT,
    user_id TEXT,
    page_id TEXT,
    text TEXT,
    embedding VECTOR(1024)
);
"""

conn = connect_to_pg()
with conn.cursor() as cur:
    cur.execute(table_ddl)
    conn.commit()
conn.close()

Insert records into our PG table. We keep appending the table (could have individual user tables but this is unnecessary in my opinion).

In [0]:
conn = connect_to_pg()
with conn.cursor() as cur:
    records = chunked_pages_pd.to_records(index=False)
    data_tuples = list(records)
    insert_query = "INSERT INTO parsed_pages (path, user_id, page_id, text, embedding) VALUES (%s, %s, %s, %s, %s)"
    cur.executemany(insert_query, data_tuples)
    conn.commit()
conn.close()

Test the table insert went through by pulling the first row. We now have a postgres table with embeddings ready to go

In [0]:
conn = connect_to_pg()
with conn.cursor() as cur:
    cur.execute("SELECT * FROM parsed_pages")
    print(cur.fetchone())
    conn.commit()
conn.close()

('dbfs:/Volumes/shm/default/raw_pdfs/73eb2c4f_3c424fe06ddfe8ce_1755574069.pdf', '7873535765378608', '0', 'Content: [# Sample Company\n## Income Statement (Service)\nFor the Year Ended September 30, 2021\n\n| Category | Amount |\n| --- | --- |\n| Service revenue | $2,750 |\n| Operating Expenses: |  |\n| Depreciation expense | 100 |\n| Wages expenses | 1,200 |\n| Supplies expenses | 60 |\n| Total operating expenses | 1,360 |\n| Operating Income | 1,390 |\n| Other Item: |  |\n| Interest expense | 40 |\n| Pretax income | 1,350 |\n| Income tax expense | 405 |\n| Net income | $945 |\n\n# Sample Company\n## Statement of Retained Earnings\nFor the Year Ended September 30, 2021\n\n| Retained Earnings | Amount |\n| --- | --- |\n| Balance, October 1, 2020 | $820 |\n| Net income | 945 |\n| Dividends declared | (500) |\n| Balance, September 30, 2021 | $1,265 |]\nFooter: []\nHeader: []\nID: [0]\nPage Number: []', '[-0.49780273,-0.27978516,-0.52441406,-0.3840332,0.28735352,0.31469727,0.3779297,-0.440

## Test Vector Search
Here is the pattern for using vector search in our app (and via LangGraph too)

In [0]:
from databricks_langchain import DatabricksEmbeddings
emb = DatabricksEmbeddings(endpoint="databricks-gte-large-en")
vect = emb.embed_documents(["hello world"])[0]

In [0]:
conn = connect_to_pg()
with conn.cursor() as cur:
    search_query = f"""
        SELECT path, user_id, page_id, text, embedding,
                (embedding <=> ARRAY{str(vect)}::vector) AS distance
        FROM parsed_pages
        ORDER BY distance ASC
        LIMIT 1
    """
    cur.execute(search_query)
    results = cur.fetchall()

In [0]:
results

[('dbfs:/Volumes/shm/default/raw_pdfs/73eb2c4f_3c424fe06ddfe8ce_1755574069.pdf',
  '7873535765378608',
  '0',
  'Content: [# Sample Company\n## Income Statement (Service)\nFor the Year Ended September 30, 2021\n\n| Category | Amount |\n| --- | --- |\n| Service revenue | $2,750 |\n| Operating Expenses: |  |\n| Depreciation expense | 100 |\n| Wages expenses | 1,200 |\n| Supplies expenses | 60 |\n| Total operating expenses | 1,360 |\n| Operating Income | 1,390 |\n| Other Item: |  |\n| Interest expense | 40 |\n| Pretax income | 1,350 |\n| Income tax expense | 405 |\n| Net income | $945 |\n\n# Sample Company\n## Statement of Retained Earnings\nFor the Year Ended September 30, 2021\n\n| Retained Earnings | Amount |\n| --- | --- |\n| Balance, October 1, 2020 | $820 |\n| Net income | 945 |\n| Dividends declared | (500) |\n| Balance, September 30, 2021 | $1,265 |]\nFooter: []\nHeader: []\nID: [0]\nPage Number: []',
  '[-0.49780273,-0.27978516,-0.52441406,-0.3840332,0.28735352,0.31469727,0.37792